In [44]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--master yarn --deploy-mode cluster --num-executors 5 --executor-memory 4g --executor-cores 1 --driver-memory 2g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "lab02_lve") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [43]:
spark.stop()

from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Normalizer
from pyspark.ml import Pipeline
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
import pyspark.sql.functions as F 
from pyspark.sql.types import DoubleType

df = spark.read.json("hdfs://spark-master-1.newprolab.com:8020/labs/slaba02/DO_record_per_line.json")
##df.show(5)
df = df.select('id','desc','lang')

tokenizer = Tokenizer(inputCol="desc", outputCol="words")
wordsData = tokenizer.transform(df)


hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

##rescaledData.select("id", "features").show()

normalizer = Normalizer(inputCol="features", outputCol="norm")
data = normalizer.transform(rescaledData)

##data_req = data.filter(data.id.isin('23126','21617', '16627','11556', '16704', '13702')==True)
data_req1 = data.filter(data.id.isin('23126')==True)
data_req2 = data.filter(data.id.isin('21617')==True)
data_req3 = data.filter(data.id.isin('16627')==True)
data_req4 = data.filter(data.id.isin('11556')==True)
data_req5 = data.filter(data.id.isin('16704')==True)
data_req6 = data.filter(data.id.isin('13702')==True)

dot_udf = F.udf(lambda x,y: float(x.dot(y)), DoubleType())
data_req1=data_req1.alias("i").join(data.alias("j"), ((F.col("i.ID") != F.col("j.ID")) &  (F.col("i.lang") == F.col("j.lang"))))\
    .select(
        F.col("i.ID").alias("i"), 
        F.col("j.ID").alias("j"), 
        dot_udf("i.norm", "j.norm").alias("dot"))\
    .limit(10).sort(F.col("dot").desc(),"i", "j")

data_req2=data_req2.alias("i").join(data.alias("j"), ((F.col("i.ID") != F.col("j.ID")) &  (F.col("i.lang") == F.col("j.lang"))))\
    .select(
        F.col("i.ID").alias("i"), 
        F.col("j.ID").alias("j"), 
        dot_udf("i.norm", "j.norm").alias("dot"))\
    .limit(10).sort(F.col("dot").desc(),"i", "j")


data_req3=data_req3.alias("i").join(data.alias("j"), ((F.col("i.ID") != F.col("j.ID")) &  (F.col("i.lang") == F.col("j.lang"))))\
    .select(
        F.col("i.ID").alias("i"), 
        F.col("j.ID").alias("j"), 
        dot_udf("i.norm", "j.norm").alias("dot"))\
    .limit(10).sort(F.col("dot").desc(),"i", "j")


data_req4=data_req4.alias("i").join(data.alias("j"), ((F.col("i.ID") != F.col("j.ID")) &  (F.col("i.lang") == F.col("j.lang"))))\
    .select(
        F.col("i.ID").alias("i"), 
        F.col("j.ID").alias("j"), 
        dot_udf("i.norm", "j.norm").alias("dot"))\
    .limit(10).sort(F.col("dot").desc(),"i", "j")


data_req5=data_req5.alias("i").join(data.alias("j"), ((F.col("i.ID") != F.col("j.ID")) &  (F.col("i.lang") == F.col("j.lang"))))\
    .select(
        F.col("i.ID").alias("i"), 
        F.col("j.ID").alias("j"), 
        dot_udf("i.norm", "j.norm").alias("dot"))\
    .limit(10).sort(F.col("dot").desc(),"i", "j")


data_req6=data_req6.alias("i").join(data.alias("j"), ((F.col("i.ID") != F.col("j.ID")) &  (F.col("i.lang") == F.col("j.lang"))))\
    .select(
        F.col("i.ID").alias("i"), 
        F.col("j.ID").alias("j"), 
        dot_udf("i.norm", "j.norm").alias("dot"))\
    .limit(10).sort(F.col("dot").desc(),"i", "j")

un_req= data_req1.union(data_req2).union(data_req3).union(data_req4).union(data_req5).union(data_req6)\
.select(F.col("i"), F.col("j"))

type(un_req.groupby("i").agg(F.collect_set("j").alias("j")).collect())


In [111]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Normalizer 
from pyspark.ml import Pipeline 
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix 
import pyspark.sql.functions as F 
from pyspark.sql.types import DoubleType

df = spark.read.json("hdfs://spark-master-1.newprolab.com:8020/labs/slaba02/DO_record_per_line.json")

##df.show(5) df = df.select('id','desc','lang')

tokenizer = Tokenizer(inputCol="desc", outputCol="words")
wordsData = tokenizer.transform(df)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000) 
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features") 
idfModel = idf.fit(featurizedData) 
rescaledData = idfModel.transform(featurizedData)

##rescaledData.select("id", "features").show()

normalizer = Normalizer(inputCol="features", outputCol="norm")
data = normalizer.transform(rescaledData)

##data_req = data.filter(data.id.isin('23126','21617', '16627','11556', '16704', '13702')==True)
data_req1 = data.filter(data.id.isin('23126')==True) 
data_req2 = data.filter(data.id.isin('21617')==True)
data_req3 = data.filter(data.id.isin('16627')==True) 
data_req4 = data.filter(data.id.isin('11556')==True)
data_req5 = data.filter(data.id.isin('16704')==True)
data_req6 = data.filter(data.id.isin('13702')==True)

dot_udf = F.udf(lambda x,y: float(x.dot(y)), DoubleType()) 
data_req1=data_req1.alias("i").join(data.alias("j"), ((F.col("i.ID") != F.col("j.ID")) & (F.col("i.lang") == F.col("j.lang"))))\
.select( F.col("i.ID").alias("i"), F.col("j.ID").alias("j"), dot_udf("i.norm", "j.norm").alias("dot"))\
.limit(10).sort(F.col("dot").desc(),"i", "j")

data_req2=data_req2.alias("i").join(data.alias("j"), ((F.col("i.ID") != F.col("j.ID")) & (F.col("i.lang") == F.col("j.lang"))))\
.select( F.col("i.ID").alias("i"), F.col("j.ID").alias("j"), dot_udf("i.norm", "j.norm").alias("dot"))\
.limit(10).sort(F.col("dot").desc(),"i", "j")

data_req3=data_req3.alias("i").join(data.alias("j"), ((F.col("i.ID") != F.col("j.ID")) & (F.col("i.lang") == F.col("j.lang"))))\
.select( F.col("i.ID").alias("i"), F.col("j.ID").alias("j"), dot_udf("i.norm", "j.norm").alias("dot"))\
.limit(10).sort(F.col("dot").desc(),"i", "j")

data_req4=data_req4.alias("i").join(data.alias("j"), ((F.col("i.ID") != F.col("j.ID")) & (F.col("i.lang") == F.col("j.lang"))))\
.select( F.col("i.ID").alias("i"), F.col("j.ID").alias("j"), dot_udf("i.norm", "j.norm").alias("dot"))\
.limit(10).sort(F.col("dot").desc(),"i", "j")

data_req5=data_req5.alias("i").join(data.alias("j"), ((F.col("i.ID") != F.col("j.ID")) & (F.col("i.lang") == F.col("j.lang"))))\
.select( F.col("i.ID").alias("i"), F.col("j.ID").alias("j"), dot_udf("i.norm", "j.norm").alias("dot"))\
.limit(10).sort(F.col("dot").desc(),"i", "j")

data_req6=data_req6.alias("i").join(data.alias("j"), ((F.col("i.ID") != F.col("j.ID")) & (F.col("i.lang") == F.col("j.lang"))))\
.select( F.col("i.ID").alias("i"), F.col("j.ID").alias("j"), dot_udf("i.norm", "j.norm").alias("dot"))\
.limit(10).sort(F.col("dot").desc(),"i", "j")

un_req= data_req1.union(data_req2).union(data_req3).union(data_req4).union(data_req5).union(data_req6)\
.select(F.col("i"), F.col("j"))

res_list=un_req.groupby("i").agg(F.collect_set("j").alias("j")).collect()



In [114]:
res = ""
res_start="{\n"
for i in res_list:
    res=res+"\""+str(i.i)+"\":"+ str(i.j) +",\n"

with open('lab02.json', 'w') as f:
    f.write(res_start+res)